In [1]:
import io

import numpy as np
import pandas as pd
from database import Database
from sklearn.base import BaseEstimator, TransformerMixin

In [2]:
bookings = pd.read_csv("bookings.csv")

db = Database()

get_hotel_mapping = db.get_hotel_mapping_as_df()
get_room_mapping = db.get_room_mapping_as_df()
get_meal_mapping = db.get_meal_mapping_as_df()
get_operator_mapping = db.get_operator_mapping_as_df()

hotel_mapping = dict(
    zip(get_hotel_mapping["external_code"], get_hotel_mapping["hotel_id"])
)
room_mapping = dict(
    zip(
        get_room_mapping.apply(lambda x: (x["external_code"], x["hotel_id"]), axis=1),
        get_room_mapping["id"],
    )
)
meal_mapping = dict(zip(get_meal_mapping["code"], get_meal_mapping["id"]))

operator_mapping = dict(
    zip(get_operator_mapping["external_code"], get_operator_mapping["operator_id"])
)

/home/jovyan/work/database.py:12: SAWarning: Did not recognize type 'geometry' of column 'geom'
  self.mapping_hotel = Table(


In [3]:
class BookingDataEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, hotels, rooms, meals, operators):
        self.hotels = hotels
        self.rooms = rooms
        self.meals = meals
        self.operators = operators

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if X is not None:
            # Operator Code
            X["operator_code"] = X["operator_code"].fillna(X["hotel_id"]).apply(str)

            # Text Case
            for col in ["guest_name", "room_code", "meal"]:
                X[col] = X[col].str.upper()

            for col in ["status", "status4", "status5"]:
                X[col] = X[col].str.capitalize()

            # Cancellation Date
            # Convert "1900-01-01" to pd.NA
            X["cancellation_date"] = X["cancellation_date"].replace("1900-01-01", pd.NA)

            # Fill missing values with "last_modified_date" where "status" is "Can"
            X.loc[
                X["cancellation_date"].isna() & (X["status"] == "Can"),
                "cancellation_date",
            ] = X["last_modified_date"]

            # Set non-matching "status" values to pd.NA
            X.loc[
                ~X["cancellation_date"].isna() & (X["status"] != "Can"),
                "cancellation_date",
            ] = pd.NA

            # Update Status
            X["status"] = X["status"].apply(lambda x: x if x == "Can" else "Ok")

            # Price Info
            X.loc[X["purchase_price"] < 1, "purchase_price"] = 0
            X.loc[X["sales_price"] < 1, "sales_price"] = 0

            # Fill the rest of blank fields with 0 if numerical
            X = X.apply(lambda x: x.fillna(0) if x.dtype.kind in "biufc" else x)

            # Hotel ID
            X["external_code_hotel"] = X["hotel_id"]
            X["hotel_id"] = pd.to_numeric(
                X["external_code_hotel"].map(self.hotels), errors="coerce"
            )
            X["hotel_id"] = X["hotel_id"].astype("Int64")

            # Room ID
            X["external_code_room"] = X["room_code"]
            X["room_id"] = X.apply(
                lambda x: self.rooms.get(
                    (
                        x["external_code_room"],
                        x["hotel_id"],
                    )
                ),
                axis=1,
            )
            X["room_id"] = pd.to_numeric(X["room_id"], errors="coerce").astype("Int64")

            # Meal ID
            X["external_code_meal"] = X["meal"]
            X["meal_id"] = X["external_code_meal"].map(self.meals)
            X["meal_id"] = pd.to_numeric(X["meal_id"], errors="coerce").astype("Int64")

            # Operator ID
            X["external_code_operator"] = X["operator_id"]
            X["operator_id"] = X["external_code_operator"].map(self.operators)
            X["operator_id"] = pd.to_numeric(X["operator_id"], errors="coerce").astype(
                "Int64"
            )

            # Fill the rest of blank fields with None for database insertion
            X.replace({pd.NaT: None, pd.NA: None, np.NaN: None}, inplace=True)

            return X
        return None

In [4]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline(
    [
        (
            "data_encoder",
            BookingDataEncoder(
                hotel_mapping,
                room_mapping,
                meal_mapping,
                operator_mapping,
            ),
        ),
    ]
)

In [5]:
df = pipeline.fit_transform(bookings)

In [6]:
df.head()

,ref_id,res_id,hotel_id,operator_id,operator_code,bkg_ref,guest_name,sales_date,in_date,out_date,...,sales_spo_code,purchase_spo_name,purchase_spo_code,main_season,external_code_hotel,external_code_room,room_id,external_code_meal,meal_id,external_code_operator
0,2060866,1758543,10000604,48,212628,501533940,NORMAN1 MORTELL1,2023-04-03,2024-01-21,2024-01-23,...,None,None,None,ALL 23-24,212628,DS,1517,BB,2,1015
1,2066493,1763570,10000624,16,DXBN04,4552834,MIKEGUENTER OKLITZ,2023-04-18,2024-03-08,2024-03-15,...,None,None,None,None,191641,DD,1582,HB+,12,69
2,2075884,1771847,10000426,16,DXB792,4560933,HANSJUERGEN STIERLIN,2023-05-16,2024-02-17,2024-03-02,...,AHGBX,BigXtra Campaign 2023/24,AHGBX,ALL 23-24,212584,DZM,967,AI,1,69
3,2077604,1773382,10000824,3,DXB04V,CA4JU,ANGIE SCHWARZ,2023-05-23,2024-01-23,2024-01-29,...,1EBD30 / 1EBD20 / 1EBD10 / 2EBD30 / 2EBD20 / 2...,Contracted Early Bird Offer,1EBD30 / 1EBD20 / 1EBD10 / 2EBD30 / 2EBD20 / 2...,ALL 23-24,133539,DXM,2081,AI,1,6
4,2156259,1773382,10000550,3,DXB384,CA4JU,ANGIE SCHWARZ,2023-05-23,2024-01-23,2024-01-29,...,LMMSBB-2305,None,None,ALL 23-24,125938,DXM,1380,AI,1,6


In [7]:
df.to_csv("pipeline.csv", index=False)

In [ ]:
import io

import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin


class BookingDataReadCsv(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if X is not None:
            return pd.read_csv(
                io.StringIO(X),
                header=0,
                names=[
                    "ref_id",
                    "res_id",
                    "hotel_id",
                    "operator_id",
                    "operator_code",
                    "bkg_ref",
                    "guest_name",
                    "sales_date",
                    "in_date",
                    "out_date",
                    "room_type",
                    "room_code",
                    "meal",
                    "days",
                    "adult",
                    "child",
                    "purchase_price",
                    "purchase_currency",
                    "sales_price",
                    "sales_currency",
                    "purchase_price_indicator",
                    "sales_price_indicator",
                    "create_date",
                    "last_modified_date",
                    "cancellation_date",
                    "status",
                    "status4",
                    "status5",
                    "purchase_contract_id",
                    "purchase_spo_id",
                    "sales_contract_id",
                    "sales_spo_id",
                    "sales_spo_name",
                    "sales_spo_code",
                    "purchase_spo_name",
                    "purchase_spo_code",
                    "main_season",
                ],
                dtype={
                    "ref_id": int,
                    "resales_id": int,
                    "hotel_id": int,
                    "operator_id": int,
                    "operator_code": str,
                    "bkg_ref": str,
                    "guest_name": str,
                    "room_type": str,
                    "room_code": str,
                    "meal": str,
                    "days": int,
                    "adult": int,
                    "child": int,
                    "purchase_price": float,
                    "purchase_currency": str,
                    "sales_price": float,
                    "sales_currency": str,
                    "purchase_price_indicator": str,
                    "sales_price_indicator": str,
                    "status": str,
                    "status4": str,
                    "status5": str,
                    "purchase_contract_id": pd.Int64Dtype(),
                    "purchase_spo_id": pd.Int64Dtype(),
                    "sales_contract_id": pd.Int64Dtype(),
                    "sales_spo_id": pd.Int64Dtype(),
                    "sales_spo_name": str,
                    "sales_spo_code": str,
                    "purchase_spo_name": str,
                    "purchase_spo_code": str,
                    "main_season": str,
                },
                parse_dates=[
                    "sales_date",
                    "in_date",
                    "out_date",
                    "create_date",
                    "last_modified_date",
                    "cancellation_date",
                ],
            )


class BookingDataEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, hotels, rooms, meals, operators):
        self.hotels = hotels
        self.rooms = rooms
        self.meals = meals
        self.operators = operators

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if X is not None:
            # Operator Code
            X["operator_code"] = X["operator_code"].fillna(X["hotel_id"]).apply(str)

            # Text Case
            for col in ["guest_name", "room_code", "meal"]:
                X[col] = X[col].str.upper()

            for col in ["status", "status4", "status5"]:
                X[col] = X[col].str.capitalize()

            # Cancellation Date
            # Convert "1900-01-01" to pd.NA
            X["cancellation_date"] = X["cancellation_date"].replace("1900-01-01", pd.NA)

            # Fill missing values with "last_modified_date" where "status" is "Can"
            X.loc[
                X["cancellation_date"].isna() & (X["status"] == "Can"),
                "cancellation_date",
            ] = X["last_modified_date"]

            # Set non-matching "status" values to pd.NA
            X.loc[
                ~X["cancellation_date"].isna() & (X["status"] != "Can"),
                "cancellation_date",
            ] = pd.NA

            # Update Status
            X["status"] = X["status"].apply(lambda x: x if x == "Can" else "Ok")

            # Price Info
            X.loc[X["purchase_price"] < 1, "purchase_price"] = 0
            X.loc[X["sales_price"] < 1, "sales_price"] = 0

            # Fill the rest of blank fields with 0 if numerical
            X = X.apply(lambda x: x.fillna(0) if x.dtype.kind in "biufc" else x)

            # Hotel ID
            X["external_code_hotel"] = X["hotel_id"]
            X["hotel_id"] = pd.to_numeric(
                X["external_code_hotel"].map(self.hotels), errors="coerce"
            )
            X["hotel_id"] = X["hotel_id"].astype("Int64")

            # Room ID
            X["external_code_room"] = X["room_code"]
            X["room_id"] = X.apply(
                lambda x: self.rooms.get(
                    (
                        x["external_code_room"],
                        x["hotel_id"],
                    )
                ),
                axis=1,
            )
            X["room_id"] = pd.to_numeric(X["room_id"], errors="coerce").astype("Int64")

            # Meal ID
            X["external_code_meal"] = X["meal"]
            X["meal_id"] = X["external_code_meal"].map(self.meals)
            X["meal_id"] = pd.to_numeric(X["meal_id"], errors="coerce").astype("Int64")

            # Operator ID
            X["external_code_operator"] = X["operator_id"]
            X["operator_id"] = X["external_code_operator"].map(self.operators)
            X["operator_id"] = pd.to_numeric(X["operator_id"], errors="coerce").astype(
                "Int64"
            )

            # Fill the rest of blank fields with None for database insertion
            X.replace({pd.NaT: None, pd.NA: None, np.NaN: None}, inplace=True)

            return X
        return None